In [ ]:
import cv2
import glob
import pandas as pd
import numpy as np
import random

# 整理輸入資料

In [ ]:
df = pd.read_csv('./cell_density_200614.csv')
df = df.loc[:,['file_name','total_cell_coverage']]
df


# 隨機看一下圖片

In [ ]:
row = random.randint(0,528)
print('row', row)
c = df.iloc[row:row+1,1].values

from keras.preprocessing.image import load_img
path = './cell_img/train_img/'+df.iloc[row]['file_name']
img = load_img(path,target_size=(150,150))
print('size:', img.size, 'coverage:',c)
img

# 建立模型

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.models import load_model, Model
from keras.layers import Flatten, Dense, Dropout, BatchNormalization

In [ ]:
resnet50 = ResNet50(include_top = False,
                   weights = 'imagenet',
                   input_shape = (150, 150, 3))

# ==================== 移植 Restnet50 (眼睛) =========================
for l in resnet50.layers:
    l.trainavle = False # 不需要 train Restnet50 中 CNN 的 weight
    
# ==================== 開始建構 MLP (大腦) =========================
x = BatchNormalization()(resnet50.output)
x = Flatten()(x)
x = Dense(units = 512, activation = 'relu')(x)
x = Dropout(0.25)(x)
x = Dense(units = 256, activation = 'relu')(x)
x = Dropout(0.25)(x)
x = Dense(units = 128, activation = 'relu')(x)
x = Dense(units = 64, activation = 'relu')(x)
x = Dense(units = 1, activation = 'sigmoid')(x)
# ==================== 合併 Restnet50 (眼睛) 和 MLP (大腦) =========================
cnn = Model (input = resnet50.input, output = x)
cnn.summary()

# Compile

In [ ]:
cnn.compile(loss = 'mse', optimizer = 'adam', metrics = ['accuracy'])

# 訓練資料的前處理

In [ ]:
np.array(path)

path_list = list(df['file_name'])
path_list
paths = ['./cell_img/train_img/'+i for i in path_list]
paths

In [ ]:
from sklearn.model_selection import train_test_split

path_list = list(df['file_name'])
paths = ['./cell_img/train_img/'+ i for i in path_list]
x_train, x_test, y_train, y_test = train_test_split(np.array(paths),
                                                   np.array(df['total_cell_coverage']),
                                                    test_size = 0.1)

# 展示 resnet50 的專用圖片前處理 (後面會再加入程式碼中)

In [ ]:
# 選擇處理圖片資訊 np 的方式, 因為 resnet50 當初論文有自己的前處理方式, 一定要按照論文的處理方式,
# 才能對應到 resnet50 的 weights

from keras.applications.resnet50 import preprocess_input
img = load_img(x_train[0])
preprocess_input(np.array(img))

In [ ]:
epoch =15 # 訓練次數
batch = 50  # 每次訓練都畫隨機抓 50 個圖進行且更新 weight

acc_track = [] # 裝每次訓練次數的 acc
loss_track = [] # 裝每次訓練 loss
time_axis = [] # 裝訓練次數, 等等畫圖用的


for each_epoch in range (epoch):
    idx = np.random.randint(0, len(x_train), 50)
    xs = []
    ys = y_train[idx]
    for b in x_train[idx]:
        img = load_img(b, target_size = (150, 150, 3))
        img = preprocess_input(np.array(img))
        xs.append(img)
    loss = cnn.train_on_batch(np.array(xs),ys )
    loss_track.append(loss[0])
    acc_track.append(loss[1])
    time_axis.append(each_epoch)
    print('================== epoch: {e}========================'.format(e = each_epoch))
    print(loss)
    